In [2]:
print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print('Importing libraries and defining some helper functions...')
from google.colab import files

print('🎉 Done!')

Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 5.6 MB 7.1 MB/s 
     |████████████████████████████████| 51 kB 6.3 MB/s 
     |████████████████████████████████| 1.4 MB 9.0 MB/s 
     |████████████████████████████████| 1.4 MB 57.6 MB/s 
     |████████████████████████████████| 204 kB 49.4 MB/s 
     |█████████████████

NameError: ignored

In [3]:
import magenta
import note_seq
import tensorflow
from note_seq.protobuf import music_pb2

print(magenta.__version__)
print(tensorflow.__version__)

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


2.1.3
2.6.0


In [4]:
# set starting pitch note on piano keyboard, you can adjust as per raga needs
pitch = 60 # C5
# 12 notes on piano keyboard
sa,ri1,ri2, ga1,ga2, ma1, ma2, pa, da1,da2, ni1,ni2, sam = pitch, pitch+1, pitch+2, pitch+3, pitch+4, pitch+5,pitch+6,pitch+7,pitch+8,pitch+9,pitch+10,pitch+11,pitch+12
# mayamalavagowla ragam notes
raga_notes = [sa, ri1, ga2, ma1, pa, da1, ni2, sam]
def play_raga(raga_notes):
  raga = music_pb2.NoteSequence()
  s_time = 0.0
# aarohana
  for swara in raga_notes:
    raga.notes.add(pitch=swara, start_time=s_time, end_time=s_time+0.5, velocity=80 )
    s_time += 0.5
  # avarohana
  for swara in raga_notes[::-1]:  # reverse notes and play as the number of notes are same as aarohana
    raga.notes.add(pitch=swara, start_time=s_time, end_time=s_time+0.5, velocity=80 )
    s_time += 0.5
  return raga 


In [6]:
# Example raga notes
# Mayamalavagowla ragam
mg = play_raga(raga_notes)
mg.total_time = 16
mg.tempos.add(qpm=60);
# This is a colab utility method that visualizes a NoteSequence.
note_seq.plot_sequence(mg)
# This is a colab utility method that plays a NoteSequence.
note_seq.play_sequence(mg,synth=note_seq.fluidsynth)
print("Mayamalavagowla [sa, ri1, ga2, ma1, pa, da1, ni2, sam] -[C,Db,E,F,G,Ab,B,C']")

# Shankarabharanam ragam
raga_notes = [sa, ri2, ga2, ma1, pa, da2, ni2, sam]
sb =play_raga(raga_notes)
sb.total_time = 16
sb.tempos.add(qpm=60);
note_seq.plot_sequence(sb)
note_seq.play_sequence(sb,synth=note_seq.fluidsynth)
print("Sankarabharanam [sa, ri2, ga2, ma1, pa, da2, ni2, sam] - [C,D,E,F,G,A,B,C']")

# Kalyani ragam
raga_notes = [sa, ri2, ga2, ma2, pa, da2, ni2, sam]
kl =play_raga(raga_notes)
kl.total_time = 16
kl.tempos.add(qpm=60);
note_seq.plot_sequence(kl)
note_seq.play_sequence(kl,synth=note_seq.fluidsynth)
print("Kalyani [sa, ri2, ga2, ma2, pa, da2, ni2, sam]-[C,D,E,F#,G,A,B,C']")

Mayamalavagowla [sa, ri1, ga2, ma1, pa, da1, ni2, sam] -[C,Db,E,F,G,Ab,B,C']


Sankarabharanam [sa, ri2, ga2, ma1, pa, da2, ni2, sam] - [C,D,E,F,G,A,B,C']


Kalyani [sa, ri2, ga2, ma2, pa, da2, ni2, sam]-[C,D,E,F#,G,A,B,C']


In [7]:
def play_sarali_swara(raga_notes):
  raga = music_pb2.NoteSequence() # returns the Note Sequence of a raga
  s_time = 0.0
  # set increment to 0.6 for 1st kalam (slow pace-1 note at a time), 
  # 0.3 for 2nd kalam (medium pace- 2 notes at a time) and 
  # 0.2 for 3rd kalam (fast pace- 4 notes at a time)
  increment = 0.3    
  vel = 50
#arohana
# 1234-32-1232-1234 pattern
  for s in range(2):  # 1st loop for aarohana(ascending notes) and 2nd loop for avarohana(descending notes)
    if s == 1:
      # reverse notes for avarohana
      raga_notes = raga_notes[::-1] 
    for i in range(5):
      for j in range(i,i+4):
        raga.notes.add(pitch=raga_notes[j], start_time=s_time, end_time=s_time+increment, velocity=vel )
        s_time += increment
      for k in range(i+2,i,-1):
        raga.notes.add(pitch=raga_notes[k], start_time=s_time, end_time=s_time+increment, velocity=vel )
        s_time += increment
      for k in range(i,i+2):
        raga.notes.add(pitch=raga_notes[k], start_time=s_time, end_time=s_time+increment, velocity=vel )
        s_time += increment
      for k in range(i+2,i,-1):
        raga.notes.add(pitch=raga_notes[k], start_time=s_time, end_time=s_time+increment, velocity=vel )
        s_time += increment
      for j in range(i,i+4):
        raga.notes.add(pitch=raga_notes[j], start_time=s_time, end_time=s_time+increment, velocity=vel )
        s_time += increment
  #return Note sequence to play
  return raga

In [8]:
# Note sequence code for sri gananadha pillari geetam (used for basic training in Carnatic Music)
# Demostrates variable length notes play
# set starting pitch note on piano keyboard
pitch= 60 # C5
# 12 (7 white and 5 black keys) notes on piano keyboard octave 
sa,ri1,ri2, ga1,ga2, ma1, ma2, pa, da1,da2, ni1,ni2 = pitch, pitch+1, pitch+2, pitch+3, pitch+4, pitch+5,pitch+6,pitch+7,pitch+8,pitch+9,pitch+10,pitch+11
sam, rim1 = pitch+12, pitch+13 # sa, ri1 of next octave

#malahari ragam 15th Melakarta Mayamalavagowla janya (child ragam) 5 in aarohana, 6 in avarohana, ni omitted
#aarohana_notes = [sa, ri1, ma1, pa, da1, sam]  # only 5 notes in aarohana (ascending notes)
#avarohana_notes = [sam, da1, pa, ma1, ga2, ri1, sa] # 6 notes in avarohana (descending notes)

#roopaka talam with 6 beats pattern

srigananadha_notes= [ma1, pa, da1, sam, sam, rim1, rim1, sam, da1, pa, ma1, pa ]
srigananadha_notes.extend([ri1, ma1, pa, da1, ma1, pa, da1, pa, ma1, ga2, ri1, sa])
srigananadha_notes.extend([sa, ri1, ma1, ga2, ri1, sa, ri1, ga2, ri1, sa])  #first ga2 and last sa need to be continued twice the length of other notes
srigananadha_notes.extend([ri1, ma1, pa, da1, ma1, pa, da1, pa, ma1, ga2, ri1, sa])

def add_note_length(raga_notes):
    length_of_notes = 1
    randomlist = []
    for swara in raga_notes:
         randomlist.append((swara, length_of_notes))
    return randomlist

#sri_gananadha_notes_with_length = add_note_length(srigananadha_notes)


# use above add_notes_length() to generate note, length tuples and set variable lengths manually, as required
sri_gananadha_notes_with_length = [(65, 1), (67, 1), (68, 1), (72, 1), (72, 1), (73, 1), (73, 1), (72, 1), (68, 1), (67, 1), (65, 1), (67, 1)]
sri_gananadha_notes_with_length.extend([(61, 1), (65, 1), (67, 1), (68, 1), (65, 1), (67, 1), (68, 1), (67, 1), (65, 1), (64, 1), (61, 1), (60, 1)])
#changed lengths to 3rd and last tuples below
sri_gananadha_notes_with_length.extend([(60, 1), (61, 1), (65, 2), (64, 1), (61, 1), (60, 1), (61, 1), (64, 1), (61, 1), (60, 2)])
sri_gananadha_notes_with_length.extend([(61, 1), (65, 1), (67, 1), (68, 1), (65, 1), (67, 1), (68, 1), (67, 1), (65, 1), (64, 1), (61, 1), (60, 1)])

def play_srigananadha(raga_notes):
  raga = music_pb2.NoteSequence()
  s_time = 0.0
  for swara in raga_notes:
    raga.notes.add(pitch=swara[0], start_time=s_time, end_time=s_time+(0.5*swara[1]), velocity=80 )
    s_time += 0.5*swara[1]
  return raga

malahari = play_srigananadha(sri_gananadha_notes_with_length)
malahari.total_time = 48

malahari.tempos.add(qpm=60);
note_seq.plot_sequence(malahari)
note_seq.play_sequence(malahari,synth=note_seq.fluidsynth)

In [9]:
# Initialize model
print('Downloading model bundle. This will take less than a minute...')
# you can change it to lookback_rnn or attention_rnn, instead of basic_rnn, as required
note_seq.notebook_utils.download_bundle('basic_rnn.mag', '/content/')

# Import dependencies.
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from note_seq.protobuf import generator_pb2
from note_seq.protobuf import music_pb2

# Initialize the model.
print("Initializing Melody RNN...")
bundle = sequence_generator_bundle.read_bundle_file('/content/basic_rnn.mag')
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map['basic_rnn'](checkpoint=None, bundle=bundle)
melody_rnn.initialize()

print('🎉 Done!')

#Continuing a sequence

#Melody_RNN generates note sequences beyond raga notes
#This is not a desired outcome pattern for raga

# Model options. Change these to get different generated sequences! 

input_sequence = malahari # change this to any raga sequence you used earlier,  if you want
num_steps = 500 # change this for shorter or longer sequences
temperature = 1.0 # the higher the temperature the more random the sequence.

# Set the start time to begin on the next step after the last note ends.
last_end_time = (max(n.end_time for n in input_sequence.notes)
                  if input_sequence.notes else 0)
qpm = input_sequence.tempos[0].qpm 
seconds_per_step = 60.0 / qpm / melody_rnn.steps_per_quarter
total_seconds = num_steps * seconds_per_step

generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = temperature
generate_section = generator_options.generate_sections.add(
  start_time=last_end_time + seconds_per_step,
  end_time=total_seconds)

# Ask the model to continue the sequence.
sequence = melody_rnn.generate(input_sequence, generator_options)

note_seq.plot_sequence(sequence)
note_seq.play_sequence(sequence, synth=note_seq.fluidsynth)

Initializing Melody RNN...
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from /tmp/tmpjs4lr314/model.ckpt
🎉 Done!
INFO:tensorflow:Beam search yields sequence with log-likelihood: -314.908203 


In [ ]:
import random
# set starting scale note on piano keyboard
pitch= 60 # C5
# 12 (7 white and 5 black keys) notes on piano keyboard octave 
sa,ri1,ri2, ga1,ga2, ma1, ma2, pa, da1,da2, ni1,ni2 = pitch, pitch+1, pitch+2, pitch+3, pitch+4, pitch+5,pitch+6,pitch+7,pitch+8,pitch+9,pitch+10,pitch+11
sam, rim1 = pitch+12, pitch+13 # sa, ri1 of next octave
# Mayamalavagowla raga notes to be used for random note generation from within the raga notes 
raga_notes = [sa, ri1, ga2, ma1, pa, da1, ni2, sam]

def generate_random_notes(raga_notes):

    no_of_notes = 10  # no of random notes to be generated
    max_length_of_notes = 1 # length of each note to be played

    randomlist = []
    for i in range(0,no_of_notes):
        n = raga_notes[random.randint(0,7)]         # generate random note in the given raga
        t = random.randint(1,max_length_of_notes)   # generate random length to play note
        randomlist.append((n,t))
    return randomlist
#send this to NodeSequence to generate raga notes
 random_notes = generate_random_notes(raga_notes)